<table>
  <tr>
    <td><img src="https://github.com/rvss-australia/RVSS/blob/main/Pics/RVSS-logo-col.med.jpg?raw=1" width="400"></td>
    <td><div align="left"><font size="30">Image Processing</font></div></td>
  </tr>
</table>

(c) Peter Corke 2024

Robotics, Vision & Control: Python, see Chapter 11

## Configuring the Jupyter environment
We need to import some packages to help us with linear algebra (`numpy`), graphics (`matplotlib`), and machine vision (`machinevisiontoolbox`).
If you're running locally you need to have these packages installed.  If you're running on CoLab we have to first install machinevisiontoolbox which is not preinstalled, this will be a bit slow.

In [ ]:
try:
    import google.colab
    print('Running on CoLab')
    !pip install machinevision-toolbox-python
    COLAB = True
except:
    COLAB = False

%matplotlib widget
import matplotlib.pyplot as plt

import numpy as np
from machinevisiontoolbox import *

# display result of assignments
if COLAB:
    %config ZMQInteractiveShell.ast_node_interactivity = 'last_expr_or_assign'
# make NumPy display a bit nicer
np.set_printoptions(linewidth=100, formatter={'float': lambda x: f"{x:10.4g}" if abs(x) > 1e-10 else f"{0:10.4g}"})


# Monadic image operations

We will work with a greyscale version of the Mona Lisa image

In [ ]:
mona = Image.Read("monalisa.png", grey=True)
mona.disp(vrange=[0,255])

First, we display some simple statistics on the minimum and maximum pixel values

In [ ]:
mona.stats()

and we see that the average pixel value is well below the mid value of 128.

To get a more nuanced idea of pixel value distribution we will compute a histogram

In [ ]:
histo = mona.hist()
histo

and plot it

In [ ]:
plt.figure()
histo.plot()

We can attempt to brighten the image by multiplying all the pixel values by 1.5.  We need to be careful that the values don't exceed the range of the `uint8` type.

In [ ]:
mona.apply(lambda x: np.uint8(np.clip(x*1.5, 0, 255))).disp(vrange=[0,255])

A simple art effect is posterization where we quantise the pixel values

In [ ]:
(mona // 64 * 64).disp(vrange=[0,255])

And we can apply a threshold, testing whether or not each pixel value exceeds 100. The resulting image has pixel values which are either True or False

In [ ]:
(mona > 100).disp()

# Green screening

This is a very common technique in video production, and these days even with video conferencing tools.  We load our foreground image, the one we'd like to superimpose on the background.  In a video this would be the presenter.

In [ ]:
foreground = Image.Read('greenscreen.png', dtype='float')
foreground.disp()

Now we need to determine which pixels are green, the background to be discarded.  We gamma decode the image and compute the chromacity of every pixel $r=R/(R+G+B), g=G/(R+G+B)$.  The result is an image with two planes

In [ ]:
cc = foreground.gamma_decode('sRGB').chromaticity()
print(cc)

We will apply a threshold to $g$ but we need to understand the distribution before we can choose the threshold value.  For that we'll compute and display a histogram.

In [ ]:
h = cc.plane('g').hist()
plt.figure()
h.plot()
plt.xlabel('Chromaticity (g)');
plt.grid(True)
ylim = plt.gca().get_ylim()


The big strong green peak on the right is the green background, the other peak is the foreground object.  A threshold of 0.45 nicely separates the two pixel populations.

In [ ]:
mask = cc.plane('g') < 0.45
mask.disp()
print(mask)


The result is a logical image where True pixels indicate foreground pixels.

We want to apply this mask to a color image so we replicate this value across the RGB color channels

In [ ]:
mask3 = mask.colorize()
print(mask3)

which we see is a color image where the R, G, and B planes are the image above.

Now we can apply the mask to the foreground image and display it.

In [ ]:
(foreground * mask3).disp()

Happily, all the green pixels have now gone.

Next we load the background image, and clip and scale it to be the same size as the foreground image.

In [ ]:
background = Image.Read("road.png", dtype="float").samesize(foreground)

and then apply the inverse mask to it.  Note that in Python `True`=1 and `False`=0.

In [ ]:
(background * (1 - mask3)).disp();

Now, all that's left to do is to add the masked foreground image to the inverse masked background image

In [ ]:
(foreground * mask3  + background * (1 - mask3)).disp();